# 1. 简单的柱状变化

下面的代码利用matplot animation 做简单的动画演示. [原文地址](http://blog.drizzt.cn/2017/08/10/matplotlib-generate-gif.html)

![快排](./perceptron.gif)


用到matplotlib.pyplot.bar函数.
~~~python
matplotlib.pyplot.bar(left, height, alpha=1, width=0.8, color=, edgecolor=, label=, lw=3)
~~~


__参数：__
1. left：x轴的位置序列，一般采用arange函数产生一个序列；
2. height：y轴的数值序列，也就是柱形图的高度，一般就是我们需要展示的数据；
3. alpha：透明度
4. width：为柱形图的宽度，一般这是为0.8即可；
5. color或facecolor：柱形图填充的颜色；
6. edgecolor：图形边缘颜色
7. label：解释每个图像代表的含义
8. linewidth or linewidths or lw：边缘or线的宽度


In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import copy

results = []

# 快速排序算法
def parttion(v, left, right):
    key = v[left]
    low = left
    high = right
    while low < high:
        while (low < high) and (v[high] >= key):
            high -= 1
        v[low] = v[high]
        while (low < high) and (v[low] <= key):
            low += 1
        v[high] = v[low]
        v[low] = key
    return low
def quicksort(v, left, right):
    if left < right:
        p = parttion(v, left, right)
        
        # 这里要采用浅拷贝方法，将每次运行结果保存到list中
        global results
        copyList = copy.copy(v)
        results.append(copyList)
        
        quicksort(v, left, p - 1)
        quicksort(v, p + 1, right)
    return v

# 设置初始值并执行算法
start = [10, 8, 1, 4, 3, 9, 5, 12, 11, 2, 7, 15, 6]
s = [10, 8, 1, 4, 3, 9, 5, 12, 11, 2, 7, 15, 6]
l = len(s)
quicksort(s, left = 0, right = l - 1)

# 画图开始
fig = plt.figure()
ax = fig.add_subplot(111) 
# 柱状图关键在于这句话
rects = plt.bar(left=(range(l)), height=(start), width=0.4, align="center", yerr=0.000001)
# 柱状图加入下标
plt.xticks((range(l)), start)

#如果是参数是list,则默认每次取list中的一个元素,results[0],results[1],...  
def update(data): 
    
    # 将上一次的柱状图清空
    plt.cla()
    
    rects = plt.bar(left=(range(l)), height=(data), width=0.4, align="center", yerr=0.000001)
    plt.xticks((range(l)), data)
    return rects

# 生成动态图的关键语句  
ani = animation.FuncAnimation(fig, update, results, interval=2*1000)  

# 利用ImageMagick导出
ani.save('perceptron.gif', fps=2, writer='imagemagick')

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# vispy: gallery 2
# Copyright (c) 2014, Vispy Development Team.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.

#
# Modified for animation with MoviePy by Zulko
# See result here: http://i.imgur.com/sSCBkFd.gif
#

"""
3D brain mesh viewer.
"""

from timeit import default_timer
import numpy as np

from vispy import gloo
from vispy import app
from vispy.util.transforms import perspective, translate, rotate
from vispy.io import load_data_file
from vispy.gloo.util import _screenshot

brain = np.load(load_data_file('brain/brain.npz', force_download='2014-09-04'))
data = brain['vertex_buffer']
faces = brain['index_buffer']

VERT_SHADER = """
#version 120
uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
uniform vec4 u_color;
attribute vec3 a_position;
attribute vec3 a_normal;
attribute vec4 a_color;
varying vec3 v_position;
varying vec3 v_normal;
varying vec4 v_color;
void main()
{
    v_normal = a_normal;
    v_position = a_position;
    v_color = a_color * u_color;
    gl_Position = u_projection * u_view * u_model * vec4(a_position,1.0);
}
"""

FRAG_SHADER = """
#version 120
uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_normal;
uniform vec3 u_light_intensity;
uniform vec3 u_light_position;
varying vec3 v_position;
varying vec3 v_normal;
varying vec4 v_color;
void main()
{
    // Calculate normal in world coordinates
    vec3 normal = normalize(u_normal * vec4(v_normal,1.0)).xyz;
    // Calculate the location of this fragment (pixel) in world coordinates
    vec3 position = vec3(u_view*u_model * vec4(v_position, 1));
    // Calculate the vector from this pixels surface to the light source
    vec3 surfaceToLight = u_light_position - position;
    // Calculate the cosine of the angle of incidence (brightness)
    float brightness = dot(normal, surfaceToLight) /
                      (length(surfaceToLight) * length(normal));
    brightness = max(min(brightness,1.0),0.0);
    // Calculate final color of the pixel, based on:
    // 1. The angle of incidence: brightness
    // 2. The color/intensities of the light: light.intensities
    // 3. The texture and texture coord: texture(tex, fragTexCoord)
    // Specular lighting.
    vec3 surfaceToCamera = vec3(0.0, 0.0, 1.0) - position;
    vec3 K = normalize(normalize(surfaceToLight) + normalize(surfaceToCamera));
    float specular = clamp(pow(abs(dot(normal, K)), 40.), 0.0, 1.0);
    
    gl_FragColor = v_color * brightness * vec4(u_light_intensity, 1);
}
"""


class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, keys='interactive')
        self.size = 400, 300

        self.program = gloo.Program(VERT_SHADER, FRAG_SHADER)
        
        self.theta, self.phi = -80, 180
        self.translate = 2.5

        self.faces = gloo.IndexBuffer(faces)
        self.program.bind(gloo.VertexBuffer(data))
        
        self.program['u_color'] = 1, 1, 1, 1
        self.program['u_light_position'] = (1., 1., 1.)
        self.program['u_light_intensity'] = (1., 1., 1.)
        
        gloo.set_state(blend=False, depth_test=True, polygon_offset_fill=True)
        self.update_matrices()

    def update_matrices(self):
        self.view = np.eye(4, dtype=np.float32)
        self.model = np.eye(4, dtype=np.float32)
        self.projection = np.eye(4, dtype=np.float32)
        
        rotate(self.model, self.theta, 1, 0, 0)
        rotate(self.model, self.phi, 0, 1, 0)
        translate(self.view, 0, 0, -self.translate)

        self.program['u_model'] = self.model
        self.program['u_view'] = self.view
        self.program['u_normal'] = np.array(np.matrix(np.dot(self.view, 
                                                             self.model)).I.T)

    def on_resize(self, event):
        width, height = event.size
        gloo.set_viewport(0, 0, width, height)
        self.projection = perspective(45.0, width / float(height), 1.0, 20.0)
        self.program['u_projection'] = self.projection


    def animation(self, t):
        """ Added for animation with MoviePy """
        self.phi, self.theta = 180*t, -80
        self.update_matrices()
        self.update()
        gloo.clear()
        self.program.draw('triangles', indices=self.faces)
        return  _screenshot((0, 0, self.size[0], self.size[1]))[:,:,:3]



if __name__ == '__main__':

    from moviepy.editor import VideoClip
    canvas = Canvas()
    canvas.show()
    clip = VideoClip(canvas.animation, duration=2)
    clip.write_videofile('brain.mp4', fps=20)
    #clip.write_gif('brain.gif', fps=20, opt='OptimizePlus', fuzz=10)

TypeError: rotate() takes from 2 to 3 positional arguments but 5 were given

In [1]:
# -*- coding: utf-8 -*-
# vispy: gallery 30
# -----------------------------------------------------------------------------
# Copyright (c) 2014, Vispy Development Team. All Rights Reserved.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.
# -----------------------------------------------------------------------------
# Author: Nicolas P .Rougier
# Date:   06/03/2014
# Abstract: Fake electrons orbiting
# Keywords: Sprites, atom, particles
#
# Modified by Zulko for animation with MoviePy
# See result here : http://i.imgur.com/6PNEYB9.gif 250 %}
# -----------------------------------------------------------------------------

import numpy as np
from vispy import gloo
from vispy import app
from vispy.util.transforms import perspective, translate, rotate
from vispy.gloo.util import _screenshot

# Create vertices
n, p = 100, 150
data = np.zeros(p * n, [('a_position', np.float32, 2),
                        ('a_color',    np.float32, 4),
                        ('a_rotation', np.float32, 4)])
trail = .5 * np.pi
data['a_position'][:, 0] = np.resize(np.linspace(0, trail, n), p * n)
data['a_position'][:, 0] += np.repeat(np.random.uniform(0, 2 * np.pi, p), n)
data['a_position'][:, 1] = np.repeat(np.linspace(0, 2 * np.pi, p), n)

data['a_color'] = 1, 1, 1, 1
data['a_color'] = np.repeat(
    np.random.uniform(0.75, 1.00, (p, 4)).astype(np.float32), n, axis=0)
data['a_color'][:, 3] = np.resize(np.linspace(0, 1, n), p * n)

data['a_rotation'] = np.repeat(
    np.random.uniform(0, 2 * np.pi, (p, 4)).astype(np.float32), n, axis=0)


vert = """
#version 120
uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
uniform float u_size;
uniform float u_clock;
attribute vec2 a_position;
attribute vec4 a_color;
attribute vec4 a_rotation;
varying vec4 v_color;
mat4 build_rotation(vec3 axis, float angle)
{
    axis = normalize(axis);
    float s = sin(angle);
    float c = cos(angle);
    float oc = 1.0 - c;
    return mat4(oc * axis.x * axis.x + c,
                oc * axis.x * axis.y - axis.z * s,
                oc * axis.z * axis.x + axis.y * s,
                0.0,
                oc * axis.x * axis.y + axis.z * s,
                oc * axis.y * axis.y + c,
                oc * axis.y * axis.z - axis.x * s,
                0.0,
                oc * axis.z * axis.x - axis.y * s,
                oc * axis.y * axis.z + axis.x * s,
                oc * axis.z * axis.z + c,
                0.0,
                0.0, 0.0, 0.0, 1.0);
}
void main (void) {
    v_color = a_color;
    float x0 = 1.5;
    float z0 = 0.0;
    float theta = a_position.x + u_clock;
    float x1 = x0*cos(theta) + z0*sin(theta);
    float y1 = 0.0;
    float z1 = (z0*cos(theta) - x0*sin(theta))/2.0;
    mat4 R = build_rotation(a_rotation.xyz, a_rotation.w);
    gl_Position = u_projection * u_view * u_model * R * vec4(x1,y1,z1,1);
    gl_PointSize = 8.0 * u_size * sqrt(v_color.a);
}
"""

frag = """
#version 120
varying vec4 v_color;
varying float v_size;
void main()
{
    float d = 2*(length(gl_PointCoord.xy - vec2(0.5,0.5)));
    gl_FragColor = vec4(v_color.rgb, v_color.a*(1-d));
}
"""

class Canvas(app.Canvas):

    def __init__(self):
        app.Canvas.__init__(self, keys='interactive')
        self.size = 800, 800

        self.program = gloo.Program(vert, frag)
        self.view = np.eye(4, dtype=np.float32)
        self.model = np.eye(4, dtype=np.float32)
        self.projection = np.eye(4, dtype=np.float32)
        self.translate = 4.5
        translate(self.view, 0, 0, -self.translate)

        self.program.bind(gloo.VertexBuffer(data))
        self.program['u_model'] = self.model
        self.program['u_view'] = self.view
        self.program['u_size'] = 5 / self.translate

        gloo.set_state('translucent', depth_test=False)
        self.program['u_clock'] = 0.0

    def on_resize(self, event):
        width, height = event.size
        gloo.set_viewport(0, 0, width, height)
        self.projection = perspective(45.0, width / float(height), 1.0, 1000.0)
        self.program['u_projection'] = self.projection

    def animation(self, t):
        """ Added for animation with MoviePy """
        self.program['u_clock'] = 2*t
        gloo.clear('black')
        self.program.draw('points')
        return  _screenshot((0, 0, self.size[0], self.size[1]))[:,:,:3]



if __name__ == '__main__':

    from moviepy.editor import VideoClip
    canvas = Canvas()
    canvas.show()
    clip = VideoClip(canvas.animation, duration=np.pi).resize(0.3)
    clip.write_videofile('atom3.mp4', fps=20)
    clip.write_gif('atom3.gif', fps=20, opt='OptimizePlus')

TypeError: translate() takes from 1 to 2 positional arguments but 4 were given

# 正弦曲线动画
![正弦曲线](./sine_wave.gif)

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')


fig = plt.figure()
ax = plt.axes(xlim=(0, 4), ylim=(-2, 2))
line, = ax.plot([], [], lw=3)

def init():
    line.set_data([], [])
    return line,
def animate(i):
    x = np.linspace(0, 4, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)

anim.save('sine_wave.gif', writer='imagemagick')

In [ ]:
# 作者：带萝卜
# 链接：https://www.zhihu.com/question/58959787/answer/361589980
# 来源：知乎
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

import matplotlib.pyplot as plt
import time

def insert_sort(lst):
    lsts = []
    for i in range(len(lst)):
        temp = lst[i]
        j = i-1
        while j>=0 and lst[j]>temp:
            lst[j+1] = lst[j]
            j -= 1
        lst[j+1] = temp
        l = lst[:]
        lsts.append(l)
    return lsts


if __name__ == "__main__":
    lst = [13,32,42,1,53,4,66,2,5,7,74,23]
    lsts = insert_sort(lst)
    plt.ion()#打开交互模式
    fig = plt.figure()#新建绘图窗口
    ax  = plt.gca()#获取当前子图
    bars = ax.bar(range(len(lst)),height=lst)#绘制条形图
    for l in lsts:
        print(l)
        bars.remove()#删除条形图
        bars = ax.bar(range(len(lst)),height=l)#绘制条形图
        plt.pause(0.5)
    while True:#防止图片关闭
        plt.pause(1)

In [15]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation

def plot(data):
    # 将上一次的柱状图清空
    plt.cla()
    n = 8
    X = np.arange(n)+1 #X是1,2,3,4,5,6,7,8,柱的个数
    #uniform均匀分布的随机数，normal是正态分布的随机数，0.5-1均匀分布的数，一共有n个
    Y1 = np.random.uniform(0.5,1.0,n)
    Y2 = np.random.uniform(0.5,1.0,n)
    plt.bar(X, Y1, alpha=0.9, width = 0.35, facecolor = 'lightskyblue', edgecolor = 'white', label='one', lw=1)
    plt.bar(X+0.35, Y2, alpha=0.9, width = 0.35, facecolor = 'yellowgreen', edgecolor = 'white', label='second', lw=1)
    plt.legend(loc="upper left") # label的位置在左上，没有这句会找不到label去哪了

plt.figure(figsize=(9,6))
# 生成动态图的关键语句  
ani = animation.FuncAnimation(plt, plot, [], interval=2*1000) 
# 利用ImageMagick导出
ani.save('perceptron2.gif', fps=2, writer='imagemagick')

AttributeError: module 'matplotlib.pyplot' has no attribute 'canvas'

<Figure size 648x432 with 0 Axes>